In [54]:
from gensim.models import KeyedVectors
import re
from tqdm import tqdm
from tqdm.contrib.concurrent import thread_map, process_map

In [2]:
w2v_model = KeyedVectors.load_word2vec_format('./model.bin', binary=True)

In [10]:
def filter_non_alpha(x):
    nonnalphachars = re.findall(r'[^A-Za-z\-]', x)
    return not bool(nonnalphachars)

In [12]:
with open('./denews70M_trigram__WordCount.txt', 'r') as f:
    d = {}
    for line in f.read().splitlines():
        k,v = line.split('\t')
        if all([int(v) >= 50, len(k) >= 5, filter_non_alpha(k)]):
            d[k] = int(v)

In [17]:
generated_dictionary = set()

In [55]:
def get_similar_candidates(word):
    if word.lower() in w2v_model:
        similar_candidates = list(map(lambda x: x[0], w2v_model.similar_by_word(word.lower(), 200)))
        similar_candidates = list(filter(lambda x: x in word, similar_candidates))
        return similar_candidates
    else:
        return []

In [56]:
generated_dictionary = process_map(get_similar_candidates, list(d.keys()), chunksize=1000, max_workers = 4)

  0%|          | 0/290917 [00:00<?, ?it/s]


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [50]:
# Produce generated dictionary
generated_dictionary = set()
for word in tqdm(list(d.keys())):
    if word.lower() in w2v_model:
        similar_candidates = list(map(lambda x: x[0], w2v_model.similar_by_word(word.lower(), 200)))
        similar_candidates = list(filter(lambda x: x in word, similar_candidates))
        generated_dictionary = generated_dictionary.union(similar_candidates)

  1%|          | 2722/290917 [18:31<32:42:00,  2.45it/s]    


KeyboardInterrupt: 

In [29]:
# Prune dictionary
for word in generated_dictionary:
    if any([word in x for x in generated_dictionary]):
        generated_dictionary.remove(word)

{'investitionen',
 'notwendigste',
 'schlimm',
 'schlimmst',
 'schlimmste',
 'verdiener',
 'wasserrohr'}

In [32]:
generated_dictionary

True